# Inspect model output

In [1]:
from datasets import load_dataset

output = load_dataset("ThatsGroes/LLM-summary-evaluation")
output

C:\Users\groes\anaconda3\envs\vllm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\groes\anaconda3\envs\vllm\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\groes\.cache\huggingface\hub\datasets--ThatsGroes--LLM-summary-evaluation. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to a

DatasetDict({
    test: Dataset({
        features: ['summary', 'dialog', 'system_prompt', 'messages', 'text', 'prompt', 'summary_by_SmolLM2-360M-Instruct-summarizer', 'summary_by_SmolLM2-1.7B-Instruct-summarizer', 'summary_by_SmolLM2-1.7B-Instruct', 'summary_by_SmolLM2-360M-Instruct'],
        num_rows: 10000
    })
})

In [12]:
output["test"]["prompt"][0]

[{'content': "Write one sentence that summarizes this conversation, emphasizing any meetings, persons or places mentioned in the conversation. \n\n **Conversation:** \n\n He's the driving force behind the Prodigy, you know. Absolutely legendary producer. Remember that track, Firestarter? Blew my mind the first time I heard it. Yeah, totally groundbreaking stuff.  They've had a few lineup changes over the years, but Howlett's always been the constant. Did you see them at Glastonbury a few years back? Epic. I think he even did a DJ set at some festival in Spain last year. Still going strong.",
  'role': 'user'}]

# Are there any identical summaries?

In [4]:
df = output["test"].to_pandas()
identicals = []
for i, row in df.iterrows():
    if len(
        set(
            [row["summary_by_SmolLM2-360M-Instruct-summarizer"].lower().strip(), 
             row["summary_by_SmolLM2-360M-Instruct"].lower().strip(), 
             row["summary_by_SmolLM2-1.7B-Instruct"].lower().strip(), 
             row["summary_by_SmolLM2-1.7B-Instruct-summarizer"].lower().strip()]
             )
    ) == 1:
        identicals.append(True)
    else:
        identicals.append(False)
        
    #print(row["summary_by_SmolLM2-360M-Instruct-summarizer"])
    #print(row["summary_by_SmolLM2-360M-Instruct-summarizer"])

In [5]:
any(identicals)

False

# Create dataset for human evaluation

In [55]:
import uuid
import json
import copy

sample = df.sample(n=100)

summary_columns = [col for col in sample.columns if "summary_by_" in col]
random.shuffle(summary_columns)

#uuids = [str(uuid.uuid1()) for i in range(len(summary_columns))]
uuids = ["a", "b", "c", "d"]

#mapping = [{"model" : model, "uuid" : uid} for model, uid in zip(summary_columns, uuids)]
mapping = {}
for model, uid in zip(summary_columns, uuids):
    mapping[model] = uid

with open('model_to_uuid_map.json', 'w') as fout:
    json.dump(mapping, fout)

# Rename columns
sample = sample.rename(columns=mapping)

# Reorder DataFrame

keep_col = ["dialog", "summary"]
keep_col.extend(uuids)
print(keep_col)
# drop cols and reorder
sample = sample[keep_col]
print(sample.head())
sample.to_csv("samples_human_eval.csv", index=False)


['dialog', 'summary', 'a', 'b', 'c', 'd']
                                                 dialog  \
3610  Write one sentence that summarizes this conver...   
4260  Write one sentence that summarizes this conver...   
2215  Write one sentence that summarizes this conver...   
578   Write one sentence that summarizes this conver...   
961   Write one sentence that summarizes this conver...   

                                                summary  \
3610  The conversation discusses the Natalia Grace c...   
4260  The conversation discusses the recent performa...   
2215  The conversation revolved around Jared Goff's ...   
578   The conversation focused on the significance o...   
961   The speakers discussed the competitive housing...   

                                                      a  \
3610  This conversation revolves around the Natalia ...   
4260  This conversation discusses the Warriors' impr...   
2215  In this conversation, the speaker discusses th...   
578   This c

In [42]:
mapping

{'summary_by_SmolLM2-1.7B-Instruct-summarizer': '51532b2d-e473-11ef-ba55-00090ffe0001',
 'summary_by_SmolLM2-1.7B-Instruct': '51532b2e-e473-11ef-81d5-00090ffe0001',
 'summary_by_SmolLM2-360M-Instruct-summarizer': '51532b2f-e473-11ef-bada-00090ffe0001',
 'summary_by_SmolLM2-360M-Instruct': '51532b30-e473-11ef-b770-00090ffe0001'}

In [45]:
sample.rename(columns=mapping)

,summary,dialog,system_prompt,messages,text,prompt,51532b2f-e473-11ef-bada-00090ffe0001,51532b2d-e473-11ef-ba55-00090ffe0001,51532b2e-e473-11ef-81d5-00090ffe0001,51532b30-e473-11ef-b770-00090ffe0001
2067,The group discussed plans to counter a Proud B...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'Write one sentence that summariz...,The group discussed plans to counter a Proud B...,The group discusses organizing a counter-prote...,We need to be prepared for the Proud Boys' ral...,We need to be prepared. The Proud Boys are hol...
4315,A discussion occurred about the growing preval...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'Write one sentence that summariz...,The conversation centered on the increasing pr...,The conversation centers around the rising pre...,This conversation discusses the growing preval...,"I mean, honestly, I feel like tipping culture ..."
3693,The conversation centers around the admiration...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'Write one sentence that summariz...,The conversation centers around the life and w...,The conversation centers around the impact and...,This conversation has been about a renowned wi...,"He's a national treasure, isn't he? I mean, wh..."
2217,The conversation centered around the shooting ...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'Write one sentence that summariz...,The conversation centered around the shooting ...,The conversation centered on the shooting of J...,This conversation discusses the senseless murd...,"It's just so senseless. I mean, seven shots in..."
9963,A group discussed the advantages and disadvant...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'Write one sentence that summariz...,The conversation focused on the pros and cons ...,The conversation focused on the advantages and...,This conversation has discussed the benefits a...,"In summary, the conversation centered around t..."
...,...,...,...,...,...,...,...,...,...,...
1879,A group of politicians discussed a plan for ec...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'Write one sentence that summariz...,A group of politicians discussed their upcomin...,The group discusses the need to pass an econom...,This conversation discusses the need to pass a...,**Summary:**\n\nWe need to focus on creating j...
2583,The conversation discussed Anthony Fauci's lon...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'Write one sentence that summariz...,The conversation discusses Anthony Fauci's ret...,The conversation centers around Anthony Fauci'...,This conversation discusses the topic of Dr. A...,I see you've summarized the conversation. Here...
2780,A discussion centered on the 1619 Project's st...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_star

# Manually inspect a random subset 

In [11]:
sample = df.sample(n=20)

for i, row in sample.iterrows():

    print(f"Dialog:{row['dialog']}\nGround truth summary:\n{row['summary']}\n")

    print("summary_by_SmolLM2-360M-Instruct-summarizer:")
    print(row["summary_by_SmolLM2-360M-Instruct-summarizer"])
    print("\n")
    print("summary_by_SmolLM2-360M-Instruct")
    print(row["summary_by_SmolLM2-360M-Instruct"])
    print("\n")
    print("summary_by_SmolLM2-1.7B-Instruct")
    print(row["summary_by_SmolLM2-1.7B-Instruct"])
    print("\n")
    print("summary_by_SmolLM2-1.7B-Instruct-summarizer")
    print(row["summary_by_SmolLM2-1.7B-Instruct-summarizer"])
    print("\n")
    print("----------------")


Dialog:Write one sentence that summarizes this conversation, emphasizing any meetings, persons or places mentioned in the conversation. 

 **Conversation:** 

 Remember that big push last month for the flu shots? What a nightmare. So many people were convinced they were gonna get microchipped. Honestly, some of the things people come up with… it's like they think we're all part of some elaborate government conspiracy. You know what I mean? And the paperwork. I swear, it took longer to document everything than it did to actually give the shot.Speaking of paperwork, have you seen the new intake forms? They’re even longer than before. It’s ridiculous, honestly. They want every detail about a patient’s life history. Did they have a pet hamster when they were five? Did their grandmother ever eat broccoli? I mean, come on.Oh, did you hear about Mrs. Henderson? The one who was in 2B last week? Apparently, she snuck out of the hospital at 3 in the morning, walked all the way downtown, and boug

# Analyse evaluation scores (BERT & ROGUE-L)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("results/evaluation_scores.csv")

In [3]:
df

,summary,dialog,system_prompt,messages,text,prompt,summary_by_SmolLM2-360M-Instruct-summarizer,summary_by_SmolLM2-1.7B-Instruct-summarizer,summary_by_SmolLM2-1.7B-Instruct,summary_by_SmolLM2-360M-Instruct,bert_score_summary_by_SmolLM2-360M-Instruct-summarizer,rougeL_score_summary_by_SmolLM2-360M-Instruct-summarizer,bert_score_summary_by_SmolLM2-1.7B-Instruct-summarizer,rougeL_score_summary_by_SmolLM2-1.7B-Instruct-summarizer,bert_score_summary_by_SmolLM2-1.7B-Instruct,rougeL_score_summary_by_SmolLM2-1.7B-Instruct,bert_score_summary_by_SmolLM2-360M-Instruct,rougeL_score_summary_by_SmolLM2-360M-Instruct
0,The conversation centers around Liam Howlett's...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'You are an expert in summarizing...,The conversation centered around the musical c...,The conversation centered around the musical c...,The conversation centered around the musical c...,The conversation centered around the musical c...,0.441898,0.333333,0.441898,0.333333,0.441898,0.333333,0.441898,0.333333
1,The speakers discuss their experiences working...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'You are an expert in summarizing...,A group of nurses discuss their experiences wo...,A group of nurses discuss their experiences wo...,A group of nurses discuss their experiences wo...,A group of nurses discuss their experiences wo...,0.427690,0.341463,0.427690,0.341463,0.427690,0.341463,0.427690,0.341463
2,This conversation revolves around a group of d...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'You are an expert in summarizing...,"A group of doctors discuss various topics, inc...","A group of doctors discuss various topics, inc...","A group of doctors discuss various topics, inc...","A group of doctors discuss various topics, inc...",0.647711,0.666667,0.647711,0.666667,0.647711,0.666667,0.647711,0.666667
3,The speakers discuss attending a meeting of th...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'You are an expert in summarizing...,The conversation centers around a planned Astr...,The conversation centers around a planned Astr...,The conversation centers around a planned Astr...,The conversation centers around a planned Astr...,0.571095,0.360000,0.571095,0.360000,0.571095,0.360000,0.571095,0.360000
4,"A group discusses a closed bakery, an upcoming...",Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'You are an expert in summarizing...,The speakers discuss the closure of a bakery o...,The speakers discuss the closure of a bakery o...,The speakers discuss the closure of a bakery o...,The speakers discuss the closure of a bakery o...,0.688221,0.575758,0.688221,0.575758,0.688221,0.575758,0.688221,0.575758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,A conversation took place about Benford's Law ...,Write one sentence that summarizes this conver...,You are an expert in summarizing texts. Extrac...,[{'content': 'You are an expert in summarizing...,<|im_start|>system\nYou are an expert in summa...,[{'content': 'You are an expert in summarizing...,"The conversation discusses Benford's Law, its ...","The conversation discusses Benford's Law, its ...","The conversation discusse

In [11]:
identicals = []
for i, row in df.iterrows():
    if len(
        set(
            [row["summary_by_SmolLM2-360M-Instruct-summarizer"], 
             row["summary_by_SmolLM2-360M-Instruct"], 
             row["summary_by_SmolLM2-1.7B-Instruct"], 
             row["summary_by_SmolLM2-1.7B-Instruct-summarizer"]]
             )
    ) == 1:
        identicals.append(True)
    else:
        identicals.append(False)
        
    #print(row["summary_by_SmolLM2-360M-Instruct-summarizer"])
    #print(row["summary_by_SmolLM2-360M-Instruct-summarizer"])

In [12]:
all(identicals)

True

In [9]:
ls = ["The conversation. Is center", "The conversation. Is center", "The Conversation. Is center"]
len(set(ls))

2